In [1]:
import os
import re
BASE_DIR='.\\analyses'

all_files = [os.path.join(BASE_DIR,file) for file in os.listdir(BASE_DIR)]


In [2]:
doc_complete=[open(file,'r',encoding='UTF-8').read() for file in all_files ]


In [3]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stopword_set = set(stopwords.words('english'))
stopword_set.update(['country','people','united','nation','peace','world','state','must','assembly','human','countries',
            'development','organization','the','nations','that','of','their','has','which','international','u','new','right','global',
                     'today','president','need','one','also','south','africa','african','soviet'])
punctuation_set = set(string.punctuation)
lemma = WordNetLemmatizer()

In [4]:
def preprocess(doc):
    remove_stopwords= " ".join(word for word in doc.lower().split() if word not in stopword_set)
    remove_punctuation = "".join(char for char in remove_stopwords if char not in punctuation_set)
    remove_digit = " ".join(word for word in remove_punctuation.split() if not word.isdigit())
    lemmatized = " ".join(lemma.lemmatize(word) for word in remove_digit.split() )
    clean= " ".join(word for word in lemmatized.lower().split() if word not in stopword_set)
    return remove_digit

In [5]:
doc_clean = [preprocess(doc).split() for doc in doc_complete] 

In [6]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [7]:
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

In [8]:
print(ldamodel.print_topics(num_topics=20, num_words=5))

[(0, '0.012*"states" + 0.006*"nuclear" + 0.005*"economic" + 0.005*"us" + 0.004*"progress"'), (1, '0.012*"states" + 0.010*"peacekeeping" + 0.010*"nuclear" + 0.009*"weapons" + 0.008*"work"'), (2, '0.009*"us" + 0.006*"strength" + 0.004*"problems" + 0.004*"power" + 0.004*"reality"'), (3, '0.006*"security" + 0.005*"general" + 0.005*"us" + 0.005*"nations" + 0.004*"council"'), (4, '0.007*"rights" + 0.006*"war" + 0.006*"first" + 0.005*"freedom" + 0.004*"time"'), (5, '0.012*"states" + 0.008*"economic" + 0.006*"growth" + 0.005*"developing" + 0.004*"future"'), (6, '0.005*"africa" + 0.005*"states" + 0.004*"us" + 0.004*"government" + 0.004*"peoples"'), (7, '0.006*"poor" + 0.006*"would" + 0.006*"us" + 0.005*"poverty" + 0.005*"general"'), (8, '0.000*"states" + 0.000*"us" + 0.000*"would" + 0.000*"nuclear" + 0.000*"general"'), (9, '0.004*"armistice" + 0.003*"korean" + 0.003*"ties" + 0.003*"confrontations" + 0.002*"greek"')]


In [9]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


Bad key text.latex.unicode in file C:\ProgramData\Anaconda3\envs\try\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.dvipnghack in file C:\ProgramData\Anaconda3\envs\try\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 127 ("text.dvipnghack : None      # some versions of dvipng don't handle alpha")
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\ProgramData\Anaconda3\envs\try\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 430 ('savefig.frameon : True')
You probably

In [10]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.127033  0.027808       1        1  37.605939
3      0.120020 -0.037130       2        1  27.977893
7      0.103239 -0.089482       3        1  12.087900
6      0.076915 -0.047941       4        1   6.942842
5      0.057964  0.053798       5        1   5.383019
1     -0.029558  0.158656       6        1   4.056912
4      0.012341  0.028758       7        1   3.391727
2     -0.146463 -0.027331       8        1   2.157134
9     -0.179576 -0.040752       9        1   0.389523
8     -0.141916 -0.026385      10        1   0.007110, topic_info=         Term        Freq       Total Category  logprob  loglift
825    states  518.000000  518.000000  Default  30.0000  30.0000
911        us  339.000000  339.000000  Default  29.0000  29.0000
1986  nuclear  260.000000  260.000000  Default  28.0000  28.0000
924       war  179.000000  179.000000  Default  27.0000  27.0000
2163  weapons  206.000000  206.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
618   peoples    0.000539  170.545684  Topic10  -9.1156  -3.1142
32     africa    0.000539  150.084116  Topic10  -9.1156  -2.9864
514      make    0.000539  144.986344  Topic10  -9.1156  -2.9519
641   poverty    0.000539  118.019292  Topic10  -9.1156  -2.7461
1255     well    0.000539  120.089115  Topic10  -9.1156  -2.7635

[818 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
7533      2  0.917745       22nd
2755      2  0.508376          a
2755      3  0.254188          a
8734      4  0.859799  abdulaziz
5964      3  0.314585    abiding
...     ...       ...        ...
6379      2  0.177128    zionism
6379      4  0.531384    zionism
6379      7  0.177128    zionism
8833      4  0.859820     ‘thank
8514      3  0.891249       “i’m

[1760 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 8, 7, 6, 2, 5, 3, 10, 9])